# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [55]:
# Dependencies and Setup
import pandas as pd

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load,encoding='iso-8859-1',low_memory=False)
student_data = pd.read_csv(student_data_to_load,encoding='iso-8859-1',low_memory=False)

# Combine the data into a single dataset (consider using a left join)
merge_df=pd.merge(school_data,student_data, on="school_name", how="left")
merge_df

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84
...,...,...,...,...,...,...,...,...,...,...,...
39165,14,Thomas High School,Charter,1635,1043130,39165,Donna Howard,F,12th,99,90
39166,14,Thomas High School,Charter,1635,1043130,39166,Dawn Bell,F,10th,95,70
39167,14,Thomas High School,Charter,1635,1043130,39167,Rebecca Tanner,F,9th,73,84
39168,14,Thomas High School,Charter,1635,1043130,39168,Desiree Kidd,F,10th,99,90


## District Summary

In [56]:
# Calculate the Totals (Schools)
total_num_schools = len(merge_df["School ID"].unique())
total_num_schools


15

In [57]:
# Calculate the Totals (Students)
total_num_students = len(merge_df["Student ID"].unique())
total_num_students

39170

In [58]:
# Calculate the Total Budget
total_budget = school_data["budget"].sum()
total_budget

24649428

In [59]:
# Calculate the Average Math
average_math_score = merge_df["math_score"].mean()
average_math_score

78.98537145774827

In [60]:
# Calculate the Average Reading Scores
average_reading_score = merge_df["reading_score"].mean()
average_reading_score

81.87784018381414

In [61]:
# Calculate the Percentage Pass Rates (students passed math)
student_data["#passing_math"] = merge_df["math_score"] >= 70
percent_passing_math = ((student_data["#passing_math"]).mean())*100
percent_passing_math


74.9808526933878

In [62]:
# Calculate the Percentage Pass Rates (students passed reading)
student_data["#passing_reading"] = merge_df["reading_score"] >= 70
percent_passing_reading = ((student_data["#passing_reading"]).mean())*100
percent_passing_reading

85.80546336482001

In [63]:
# Calculate the Percentage Pass Rates (overall passed)
overall_passing_rate = (percent_passing_math + percent_passing_reading)/2
overall_passing_rate


80.39315802910392

In [64]:
# Minor Data Cleanup
district_results = [{"Total Schools": total_num_schools, 
                     "Total Students": total_num_students, 
                     "Total Budget": total_budget, 
                     "Average Math Score":  round(average_math_score,2), 
                     "Average Reading Score":  round(average_reading_score,2), 
                     "% Passing Math": round(percent_passing_math,2),
                     "% Passing Reading": round(percent_passing_reading,2),
                     "% Overall Passing Rate": round(overall_passing_rate,2)}]
district_summary_table = pd.DataFrame(district_results)

district_summary_table["% Passing Math"] = district_summary_table["% Passing Math"].map("{:,.2f}%".format)
district_summary_table["% Passing Reading"] = district_summary_table["% Passing Reading"].map("{:,.2f}%".format)
district_summary_table["% Overall Passing Rate"] = district_summary_table["% Overall Passing Rate"].map("{:,.2f}%".format)
district_summary_table["Total Budget"] = district_summary_table["Total Budget"].map("${:,.2f}".format)
district_summary_table["Total Students"] = district_summary_table["Total Students"].map("{:,}".format)

district_summary_table


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.39%


## School Summary

In [65]:
# Determine the School Type
school_type=merge_df.groupby(['school_name'])
school_type

In [66]:
# Calculate the total student count
student_count = merge_df["school_name"].value_counts()

student_count

Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: school_name, dtype: int64

In [67]:
# Calculate the total school budget and per capita spending

per_school_budget=merge_df.groupby(["school_name"]).mean()["budget"]
student_budget=per_school_budget/student_count
student_budget


Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [68]:
# Calculate the average test scores
avg_math = merge_df.groupby(["school_name"]).mean()['math_score']
avg_math

school_name
Bailey High School       77.048432
Cabrera High School      83.061895
Figueroa High School     76.711767
Ford High School         77.102592
Griffin High School      83.351499
Hernandez High School    77.289752
Holden High School       83.803279
Huang High School        76.629414
Johnson High School      77.072464
Pena High School         83.839917
Rodriguez High School    76.842711
Shelton High School      83.359455
Thomas High School       83.418349
Wilson High School       83.274201
Wright High School       83.682222
Name: math_score, dtype: float64

In [69]:
# Calculate the average test scores
avg_reading = merge_df.groupby(["school_name"]).mean()['reading_score']
avg_reading

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [70]:
# Calculate the passing scores by creating a filtered data frame
passing_math = merge_df[(merge_df["math_score"] >= 70)]
passing_math=passing_math.groupby(["school_name"]).count()['student_name']/student_count*100


passing_reading = merge_df[(merge_df["reading_score"] >= 70)]
passing_reading=passing_reading.groupby(["school_name"]).count()['student_name']/student_count*100

over_all=(passing_math + passing_reading)/2
over_all


Bailey High School       74.306672
Cabrera High School      95.586652
Figueroa High School     73.363852
Ford High School         73.804308
Griffin High School      95.265668
Hernandez High School    73.807983
Holden High School       94.379391
Huang High School        73.500171
Johnson High School      73.639992
Pena High School         95.270270
Rodriguez High School    73.293323
Shelton High School      94.860875
Thomas High School       95.290520
Wilson High School       95.203679
Wright High School       94.972222
dtype: float64

In [71]:
# Convert to data frame
school_summary=pd.DataFrame ({                                  
                      "Total Students": student_count,
                      "Total School Budget": per_school_budget,
                      "Per Student Budget": student_budget,
                      "Average Math Score": round(avg_math,2),
                      "Average Reading Score":round(avg_reading,2),
                      "% Passing Math": passing_math,
                      "% Passing Reading": passing_reading,
                      "% Overall Passing Rate": over_all})




school_summary["Total Students"]=school_summary["Total Students"].map("{:,.0f}".format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)






school_summary

,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.680064,81.933280,74.306672
Cabrera High School,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.133477,97.039828,95.586652
Figueroa High School,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.988471,80.739234,73.363852
Ford High School,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.309602,79.299014,73.804308
Griffin High School,"1,468","$917,500.00",$625.00,83.35,83.82,93.392371,97.138965,95.265668
Hernandez High School,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.752967,80.862999,73.807983
Holden High School,427,"$248,087.00",$581.00,83.80,83.81,92.505855,96.252927,94.379391
Huang High School,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.683922,81.316421,73.500171
Johnson High School,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.057551,81.222432,73.639992
Pena High School,962,"$585,858.00",$609.00,83.84,84.04,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

In [72]:
# Sort and show top five schools
top_performing=school_summary.sort_values(by="% Overall Passing Rate",ascending=False)
top_performing


,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.133477,97.039828,95.586652
Thomas High School,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.272171,97.308869,95.290520
Pena High School,962,"$585,858.00",$609.00,83.84,84.04,94.594595,95.945946,95.270270
Griffin High School,"1,468","$917,500.00",$625.00,83.35,83.82,93.392371,97.138965,95.265668
Wilson High School,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.867718,96.539641,95.203679
Wright High School,"1,800","$1,049,400.00",$583.00,83.68,83.96,93.333333,96.611111,94.972222
Shelton High School,"1,761","$1,056,600.00",$600.00,83.36,83.73,93.867121,95.854628,94.860875
Holden High School,427,"$248,087.00",$581.00,83.80,83.81,92.505855,96.252927,94.379391
Bailey High School,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.680064,81.933280,74.306672
Hernandez High School,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.752967,80.862999,73.807983


## Bottom Performing Schools (By Passing Rate)

In [73]:
# Sort and show bottom five schools

bottom_performing=school_summary.sort_values(by="% Overall Passing Rate")
bottom_performing


,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.366592,80.220055,73.293323
Figueroa High School,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.988471,80.739234,73.363852
Huang High School,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.683922,81.316421,73.500171
Johnson High School,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.057551,81.222432,73.639992
Ford High School,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.309602,79.299014,73.804308
Hernandez High School,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.752967,80.862999,73.807983
Bailey High School,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.680064,81.933280,74.306672
Holden High School,427,"$248,087.00",$581.00,83.80,83.81,92.505855,96.252927,94.379391
Shelton High School,"1,761","$1,056,600.00",$600.00,83.36,83.73,93.867121,95.854628,94.860875
Wright High School,"1,800","$1,049,400.00",$583.00,83.68,83.96,93.333333,96.611111,94.972222


## Math Scores by Grade

In [74]:
# Create data series of scores by grade levels using conditionals
ninth_graders = merge_df[(merge_df["grade"] == "9th")]
tenth_graders = merge_df[(merge_df["grade"] == "10th")]
eleventh_graders = merge_df[(merge_df["grade"] == "11th")]
twelfth_graders = merge_df[(merge_df["grade"] == "12th")]
# Group each by school name
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]
# Combine series into single data frame
scores_by_grade = pd.DataFrame({"9th": ninth_graders_scores, "10th": tenth_graders_scores,
                                "11th": eleventh_graders_scores, "12th": twelfth_graders_scores})
scores_by_grade
# Minor data munging

# Display the data frame


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [75]:
# Create data series of scores by grade levels using conditionals
ninth_graders = merge_df[(merge_df["grade"] == "9th")]
tenth_graders = merge_df[(merge_df["grade"] == "10th")]
eleventh_graders = merge_df[(merge_df["grade"] == "11th")]
twelfth_graders = merge_df[(merge_df["grade"] == "12th")]

# Group each by school name
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]
# Combine series into single data frame
scores_by_grade = pd.DataFrame({"9th": ninth_graders_scores, "10th": tenth_graders_scores,
                                "11th": eleventh_graders_scores, "12th": twelfth_graders_scores})
scores_by_grade
# Minor data munging

# Display the data frame


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [76]:
# Establish the bins -- choose any set of bins you would like, but see below for testing bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
# to test, set your bins as follows: [0, 585, 615, 645, 675]

# ALSO -- Note that the values for `% Passing Math`, `% Passing Reading` and `% Overall Passing Rate`
# were computed using averages of averages -- your results may vary if you use weighted averages 

# Categorize the spending based on the bins
school_summary["Spending Ranges (Per Student)"] = pd.cut(student_budget, spending_bins, labels=group_names)

spending_math_scores = school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_rate = (spending_math_scores + spending_reading_scores) / 2
# Assemble into data frame
spending_summary = pd.DataFrame({"Average Math Score" : spending_math_scores,
                                 "Average Reading Score": spending_reading_scores,
                                 "% Passing Math": spending_passing_math,
                                 "% Passing Reading": spending_passing_reading,
                                 "% Overall Passing Rate": overall_passing_rate})

# Minor data munging

# Display results
spending_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.452500,83.935000,93.460096,96.610877,83.693750
$585-615,83.600000,83.885000,94.230858,95.900287,83.742500
$615-645,79.078333,81.891667,75.668212,86.106569,80.485000
$645-675,76.996667,81.026667,66.164813,81.133951,79.011667


## Scores by School Size

In [81]:
# Establish the bins 
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


# Categorize the spending based on the bins
merge_df['size_bins'] = pd.cut(merge_df['size'], size_bins, labels = group_names)
by_size = merge_df.groupby('size_bins')
# Calculate the scores based on bins

avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = merge_df[merge_df['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
pass_read = merge_df[merge_df['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
overall = merge_df[(merge_df['reading_score'] >= 70) & (merge_df['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()

# Assemble into data frame
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
})
# Minor data munging
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]    
scores_by_size.index.name = "Total Students"
scores_by_size = scores_by_size.reindex(group_names)

# Display results
scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,
Small (<1000),83.828654,83.828654,0.939525,0.960403,0.901368
Medium (1000-2000),83.372682,83.372682,0.936165,0.967731,0.906243
Large (2000-5000),77.477597,77.477597,0.686524,0.821252,0.565740


## Scores by School Type

In [85]:
# Type | Average Math Score | Average Reading Score | % Passing Math | % Passing Reading | % Overall Passing Rate
by_type = merge_df.groupby("type")


avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = merge_df[merge_df['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
pass_read = merge_df[merge_df['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
overall = merge_df[(merge_df['reading_score'] >= 70) & (merge_df['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()

# Assemble into data frame
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})

# Minor data munging

# Display result
scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.406183,83.406183,0.937018,0.966459,0.905609
District,76.987026,76.987026,0.665184,0.809052,0.536959
